# WhisperTS joining JSON to SRT

In [33]:
import re,codecs,unidecode
import pandas as pd
import networkx as nx
import matplotlib.pylab as p

import json,glob,os
nMachine=os.uname()[1]
if "iMac" in nMachine:
    nMachine="iMac"
elif "MacBook" in nMachine:
    nMachine="MacBook"
print(nMachine)

iMac


In [34]:
repJson="/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/%s/Json/"%nMachine
segments=len(glob.glob(repJson+"*.json"))
segments

11

In [35]:
json1=repJson+"extrait-00.wav.json"
with open(json1) as inFile:
    whisper1=json.load(inFile)

### Gestion des timestamps

In [36]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

def shiftTime(timestamp,decal):
    return second2timestamp(timestamp2second(timestamp)+decal)

In [37]:
punct = list('''!()[]{};:-'"\,<>./?@#$%^&*_~''')
maxSize=8

def removePunct(texte):
    result=texte
    for p in punct:
        result=result.replace(p,"")
    return result

### Gestion des structures JSON de Whisper

In [38]:
def getWords(whisper):
    lWords=[]
    for segment in whisper["segments"]:
        for word in segment["words"]:
            lWords.append((second2timestamp(word["start"]),
                              second2timestamp(word["end"]),
                              word["text"],
                              word["confidence"]))        
    return [(t1,t2,w,c) for t1,t2,w,c in lWords if w!=""]

def getList(tWhisper):
    return [unidecode.unidecode(removePunct(w)).lower() for _,_,w,_ in tWhisper]

In [39]:
srt=getWords(whisper1)
srtList=getList(srt)

### Trouver la jointure

In [40]:
def findSample(sample,ref):
    i=0
    while i<len(ref)-len(sample):
        # print("ref",i)
        if sample==ref[i:i+len(sample)]:
            return i
        else:
            i+=1
    return None

### Collage des mots
1. on cherche la partie commune entre le nouveau segment et les sous-titres précédents
2. on peut régler la longueur de la partie commune pour optimiser le collage *maxSize*
3. on incrémente les temps de la nouvelle partie à partir du deuxième mot (pour éviter les erreurs de timestamp en début d'enregistrement)

In [41]:
def pasteWords(srt,whisper2):
    srtList=getList(srt)
    w2=getWords(whisper2)
    w2List=getList(w2)
    lenW2=len(w2)
    compSize=min(maxSize,lenW2)
    i=1
    print(compSize)
    while i<lenW2-compSize:
        sample=w2List[i:i+compSize]
        ref=srtList
        # print(sample)
        idx=findSample(sample,ref)
        if idx:
            print("found",i,idx)
            break
        else:
            i+=1
    if idx:
        print("found",i,idx,srtList[idx],srt[idx][0],w2List[i],w2[i][0])
        tDecal=timestamp2second(srt[idx+1][0])-timestamp2second(w2[i+1][0])
        print(second2timestamp(tDecal))
        newSrt=[]
        for n in range(i+1,lenW2):
            nWord=w2[n]
            newWord=(shiftTime(nWord[0],tDecal),shiftTime(nWord[1],tDecal),nWord[2],nWord[3])
            newSrt.append(newWord)
        print(srt[idx+1],newSrt[0])
        return srt[:idx]+newSrt
    else:
        return None

In [42]:
if segments:
    for i in range(1,segments):
        print(i)
        json2=repJson+"extrait-%02d.wav.json"%i
        with open(json2) as inFile:
            whisper2=json.load(inFile)
        newSrt=pasteWords(srt,whisper2)
        if not newSrt:
            break
        else:
            srt=newSrt
# print(srt)

1
8
found 7 86
found 7 86 estce 00:01:03,560 estce 00:00:03,759
00:01:00,001
('00:01:04,180', '00:01:04,219', 'que', 1.0) ('00:01:04,180', '00:01:04,201', 'que', 0.976)
2
8
found 1 212
found 1 212 est 00:01:54,540 est 00:00:01,960
00:01:52,560
('00:01:54,680', '00:01:54,921', 'où ?', 0.943) ('00:01:54,680', '00:01:54,978', 'où ?', 0.703)
3
8
found 5 326
found 5 326 mais 00:02:54,359 mais 00:00:01,760
00:02:52,459
('00:02:54,379', '00:02:54,778', 'non,', 0.983) ('00:02:54,378', '00:02:54,679', 'non,', 0.872)
4
8
found 1 506
found 1 506 toi 00:03:53,239 toi 00:00:00,680
00:03:52,518
('00:03:53,398', '00:03:53,519', "t'es", 0.999) ('00:03:53,397', '00:03:53,578', "t'es", 0.89)
5
8
found 2 695
found 2 695 ah 00:04:53,838 ah 00:00:01,000
00:04:52,638
('00:04:54,018', '00:04:54,317', 'ouais ?', 1.0) ('00:04:54,017', '00:04:54,418', 'ouais ?', 0.721)
6
8
found 4 871
found 4 871 pas 00:05:56,158 pas 00:00:03,540
00:05:52,588
('00:05:56,208', '00:05:56,418', "d'avoir", 0.999) ('00:05:56,208', '

In [43]:
pause=0.200

srtTours=[]
srtMots=[]
bTour=True
prevEnd=0
pStart=second2timestamp(0)
pEnd=0

for i,(start,end,mot,confidence) in enumerate(srt):
    # print(start,end,mot,confidence)
    tStart=timestamp2second(start)
    tEnd=timestamp2second(end)
    # print(start,tStart,prevEnd)
    if tStart-prevEnd>pause and prevEnd!=0:
        pEnd=second2timestamp(prevEnd)
        pTour=" ".join(srtMots)
        srtTours.append((pStart,pEnd,pTour))
        pStart=start
        prevEnd=tEnd
        srtMots=[mot]
        bPhrase=True
    else:
        srtMots.append(mot)
        prevEnd=tEnd
pTour=" ".join(srtMots)
srtTours.append((pStart,pEnd,pTour))

In [44]:
srtLines=[]
for i,(start,end,mot) in enumerate(srtTours):
    print(start,end,mot)
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


00:00:00,000 00:00:06,780 [*] Ok, bon, [*] écoutez, on va essayer de pécho autre part, puisqu'elle ne veut pas de nous.
00:00:07,679 00:00:18,359 Oh, mais coucou toi ! [*] Présentation [*] amicale, on va faire ça, on va tenter, on va se [*] présenter à cette jeune demoiselle.
00:00:20,179 00:00:21,940 Ah oui, je pense qu'il a le seum, effectivement.
00:00:23,980 00:00:26,859 Fous-lui une claque. Ah non, je ne peux pas, je ne peux pas, ce n'est pas possible.
00:00:56,979 00:00:59,320 Bjorn ? Elle est une youtubeuse, mais en même temps, elle est tellement...
00:01:00,159 00:01:02,060 Merci, Mélodie, c'est très gentil.
00:01:03,259 00:01:03,539 Oh,
00:01:04,180 00:01:05,700 que nous allons enfin pouvoir pécho ?
00:01:06,140 00:01:08,420 Allons-nous avoir une touche avec cette jeune Alice ?
00:01:09,439 00:01:12,081 Embrasser les mains, peut-être pas, c'est peut-être un petit peu trop rapide.
00:01:12,780 00:01:15,381 Peut-on faire des photos [*] avec cette jeune Alice ?
00:01:16,060 00:01

In [45]:
with codecs.open(json1.replace(".json","-tours.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")